# Neural Networks


[Tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [39]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

## Data

Load the Data using pandas/numpy:

In [40]:
df=pd.read_csv('./DataSets/winequality-red.csv', sep=';')
df=np.array(df.values)

The last column are the features so extract those:

In [41]:
y = df[:,-1]
X = df[:,range(df.shape[1]-1)]

In [42]:
y = y>5

Finally make sure that these are tensors

In [43]:
X = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(y.astype(np.float32))

## Torch Class

Define a torch class, in this case we'll use an 11-4-1, where the output measures whether or not the wine is good.

In [44]:

class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        # Create the input layer and the hidden layer
        self.hidden1 = nn.Linear(11, 10)
        self.hidden2 = nn.Linear(10, 8)
        self.hidden3 = nn.Linear(8, 4)
        self.output = nn.Linear(4, 1)

        # Define the activation functions that will be used
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1) # dim=1 calculates softmax across cols

    def forward(self, x):
        # Take input
        x = self.hidden1(x)  # Linear Combination of input-> hidden
        x = self.hidden2(x)  # Linear Combination of input-> hidden
        x = self.hidden3(x)  # Linear Combination of input-> hidden
        x = self.sigmoid(x) # Activation Function
        x = self.output(x)  # Linear Combination of hidden -> output
        x = self.sigmoid(x) # Activation Function

        return x

# Assign the model object
net = Network()
print(net)

Network(
  (hidden1): Linear(in_features=11, out_features=10, bias=True)
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (hidden3): Linear(in_features=8, out_features=4, bias=True)
  (output): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
)


### Generate the Model Output

In [45]:

## Print the Model Output
out = net(X)
print(out)

tensor([[0.4697],
        [0.4705],
        [0.4639],
        ...,
        [0.4759],
        [0.4758],
        [0.4696]], grad_fn=<SigmoidBackward>)


## Measure the Loss Function

First define a loss function,  We'll use *MSE* here, but there are [many others](https://pytorch.org/docs/stable/nn.html#loss-functions). Also define the form of Gradient Descent implemented

In [46]:
eta = 1/1000

import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=eta, momentum = 0.9)

## Train the Network

Now that gradient descent model, the structure of the network and the loss function are all defined we can begin training the network.

In [47]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss=0
    for i in range(X.shape[0]):
        # Get the input and desired output
        input   = X[i,:]
        target  = y[i]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward, then backward, then optimize
        ## Calculate output
        outputs = net(input)
        ## Measure the Loss
        loss   = criterion(outputs, target)
        ## Calculate the Gradients
        loss.backward()
        ## Adjust the weights
        optimizer.step()

        ## Print any Statistics
        running_loss += loss.item()
        # print(loss.item())
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

   
print('Finished Training')


[1,   200] loss: 0.022
[1,   400] loss: 0.026
[1,   600] loss: 0.024
[1,   800] loss: 0.025
[1,  1000] loss: 0.025
[1,  1200] loss: 0.020
[1,  1400] loss: 0.026
[2,   200] loss: 0.022
[2,   400] loss: 0.024
[2,   600] loss: 0.024
[2,   800] loss: 0.025
[2,  1000] loss: 0.023
[2,  1200] loss: 0.020
[2,  1400] loss: 0.026
Finished Training


### Save the model

In [48]:
PATH = './wine_neural_network.pth'
torch.save(net.state_dict(), PATH)

## Print the Parameters of the Neural Network

In [49]:
for name, param in net.named_parameters():
    if param.requires_grad:
        print(name, param.data)


hidden1.weight tensor([[-2.9496e-01, -1.8367e-01,  1.7613e-01,  7.1353e-04, -2.0044e-01,
         -2.2062e-01, -1.3609e-01, -1.1500e-01,  1.5059e-01,  1.4561e-01,
         -3.4647e-01],
        [-1.1676e-01, -5.9055e-02,  2.0699e-01,  1.7329e-01, -1.0591e-01,
         -4.9562e-02, -2.7079e-01, -6.5656e-02,  2.9439e-01,  1.0861e-01,
          2.3687e-01],
        [ 2.1225e-01,  1.7519e-01,  1.7001e-01, -1.3865e-04, -1.8732e-01,
         -2.1608e-01, -2.2757e-02, -7.4580e-02,  2.6850e-01,  2.5534e-01,
          9.3617e-02],
        [-2.4720e-01, -2.6311e-02, -2.0202e-01,  2.3485e-02,  2.1087e-01,
         -3.3093e-02, -5.6192e-02,  1.7933e-01, -2.1967e-01,  1.6400e-01,
         -2.1926e-01],
        [ 1.8310e-01, -2.8201e-01,  3.1032e-02,  2.9673e-01,  2.2340e-01,
         -2.7925e-02, -3.1455e-01, -1.9614e-01, -9.2661e-02,  1.8298e-01,
         -2.4908e-02],
        [-7.0997e-02, -8.3039e-02,  1.1082e-01,  2.0215e-01,  2.3935e-01,
          2.4391e-01,  2.4319e-01,  2.1143e-01, -3.2762e

## Print the Misclassification Rate

In [87]:
yhat = net(X)
yhat = yhat.detach().numpy().reshape(-1) > 0.3
print(yhat)

[ True  True  True ...  True  True  True]


In [88]:
y = df[:,-1]
y = y>5
np.array(y)

array([False, False, False, ...,  True, False,  True])

In [89]:
correctQ = [ i==j for i in yhat for j in y ]

In [90]:
print(np.average(correctQ))

0.5347091932457786


Hmm, so even though the loss is quite low, it seems that in misclassificatoin rate is not very good.